In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from image_utils import *
from pathlib import Path
import datetime
import matplotlib.pyplot as plt
import numpy as np
import skimage
from scipy import ndimage


def get_date(path):
    """Date from path"""

    try:
        date = str(path).split('\\')[-1].split('_')[0]
        year = int(date[:4])
        month = int(date[4:6])
        day = int(date[6:])
        date = datetime.date(year, month, day)
    except:
        return None

    return date

def show_dif(img, date):
    fig, ax = plt.subplots(1,3, figsize = (9,3))
    ax[0].imshow(img[0], cmap = 'gray_r')
    ax[1].imshow(img[1], cmap = 'gray_r')
    ax[2].imshow(img[0]/np.max(img[0]) - img[1]/np.max(img[1]), cmap = 'gray_r')
    plt.suptitle(date)
    plt.show()


def upscale(image, scale):
    """Upscale image without interpolation and anti-aliasing."""

    upscaled_image = skimage.transform.rescale(image, scale, preserve_range = True, order = 0, anti_aliasing = False)

    return upscaled_image

def find_peaks(image, threshold, box_size, stack_number = None):
    """Find local maxima in 2D image."""

    p = box_size // 2
    peaks = []

    itemindex = np.where(image > threshold)

    for i,j in zip(itemindex[0], itemindex[1]):
        if i > p and j > p:
            crop = image[i-p:i+p+1,j-p:j+p+1]
            if np.unravel_index(np.argmax(crop), (p*2+1,p*2+1)) == (p,p):
                if stack_number == None:
                    peaks.append([i,j,image[i,j]])
                else:
                    peaks.append([i,j,image[i,j],stack_number])
    return peaks

def colormap(image, color, scale=None):
    '''Changes the colormap of the grayscale image (2D) to one of the six basic colors.'''

    h, w = image.shape
    result = np.zeros((h, w, 3))

    if scale == None:
        norm_image = image/np.max(image)
    else:
        norm_image = image/scale

    if color == 'red' or color == 'magenta' or color == 'yellow' or color == 'white' or color == 'gray':
        result[:, :, 0] = norm_image
    if color == 'green' or color == 'cyan' or color == 'yellow' or color == 'white' or color == 'gray':
        result[:, :, 1] = norm_image
    if color == 'blue' or color == 'magenta' or color == 'cyan' or color == 'white' or color == 'gray':
        result[:, :, 2] = norm_image

    result = np.clip(result, 0, 1)

    return result

def misalignment(path, config, disp= False, coord_average = False):


    if type(config) == list:
        for c in config:
            try:
                settings, img = read_msr_configs(path, config=c)
            except:
                pass
    else:
        settings, img = read_msr_configs(path, config=config)

    pixel_size = int(settings.pixel_sizes['X'][0]*1e9)
    upsc = pixel_size // 10

    date = get_date(path)
    labels = img.labels()
    imp = img.data('CYX')
    dist = 20
   
    imgt = upscale(imp[0], upsc)
    simg = ndimage.gaussian_filter(imgt, sigma=3 * upsc, output = float)
    peaks1 = find_peaks(simg, np.max(imp[0]*.1), 20 * upsc)
    peaks_x1 = np.asarray(peaks1)[:,0]
    peaks_y1 = np.asarray(peaks1)[:,1]

    imgt = upscale(imp[1], upsc)
    simg = ndimage.gaussian_filter(imgt, sigma=3, output = float)
    peaks2 = find_peaks(simg, np.max(imp[1]*.1), 20 * upsc)
    peaks_x2 = np.asarray(peaks2)[:,0]
    peaks_y2 = np.asarray(peaks2)[:,1]

    q = []
    for x1, y1, in zip(peaks_x1, peaks_y1):
        for x2, y2 in zip(peaks_x2, peaks_y2):
            if np.linalg.norm([x1-x2, y1-y2]) < dist:
                q.append([x1,y1,x2,y2, np.linalg.norm([x1-x2, y1-y2])])  
    sets_array = np.asarray(q)

    x1, y1, x2, y2, d = np.mean(q, axis = 0)
    if coord_average:
        d = np.linalg.norm([x1-x2, y1-y2])
    d = d/upsc * pixel_size
    xdiff = (x1-x2)/upsc * pixel_size
    ydiff = (y1-y2)/upsc * pixel_size


    if disp:
        fig, ax = plt.subplots(1,3, figsize = (12,4))
    
        ax[0].imshow(imp[0], cmap = 'gray') 
        ax[0].scatter(peaks_y1/upsc, peaks_x1/upsc, color = 'red', s = 2)
        ax[0].axis('off')
        ax[0].set_title(labels[0])

        ax[1].imshow(imp[1], cmap = 'gray') 
        ax[1].scatter(peaks_y2/upsc, peaks_x2/upsc, color = 'red', s = 2)
        ax[1].axis('off')
        ax[1].set_title(labels[1])

        ax[2].imshow(colormap(imp[0], 'magenta') + colormap(imp[1], 'green'))
        ax[2].scatter(sets_array[:,1]/upsc, sets_array[:,0]/upsc, alpha = 1, color = 'cyan', s = 2)
        ax[2].scatter(sets_array[:,3]/upsc, sets_array[:,2]/upsc, alpha = 1, color = 'cyan', s = 2)
        ax[2].axis('off')
        ax[2].set_title('Merge')
    
        plt.suptitle(f'{date} - {np.linalg.norm([xdiff, ydiff]):.1f} nm')

        plt.savefig(f'misalignment/{date}.png')
        plt.close()


    return xdiff, ydiff, d, date

def get_hists(img):

    fig, ax = plt.subplots(2,3, figsize = (8, 4))
    ax = ax.flatten()
    
    flat0 = img[0].flatten()
    ax[0].hist(flat0.flatten(), bins = 100)
    ax[1].hist(flat0[flat0 > 5])
    ax[1].set_title('Channel1')
    ax[2].hist(flat0[flat0 > 20])
    
    flat1 = img[1].flatten()
    ax[3].hist(flat1.flatten(), bins = 100)
    ax[4].hist(flat1[flat1 > 5])
    ax[4].set_title('Channel2')
    ax[5].hist(flat1[flat1 > 20])
    
    plt.tight_layout()
    plt.show()


def align(path, config):
    settings, img = read_msr_configs(path, config=config)
    pixel_size = int(settings.pixel_sizes['X'][0]*1e9)
    upsc = pixel_size//10
    
    date = get_date(path)
    labels = img.labels()
    imp = img.data('CYX')
    dist = 20
    imgt = upscale(imp[0], upsc)
    simg1 = ndimage.gaussian_filter(imgt, sigma=3 * upsc, output = float)
    peaks1 = find_peaks(simg1, np.max(imp[0]*.1), 20 * upsc)
    peaks_x1 = np.asarray(peaks1)[:,0]
    peaks_y1 = np.asarray(peaks1)[:,1]
    
    imgt = upscale(imp[1], upsc)
    simg2 = ndimage.gaussian_filter(imgt, sigma=3, output = float)
    peaks2 = find_peaks(simg2, np.max(imp[1]*.1), 20 * upsc)
    peaks_x2 = np.asarray(peaks2)[:,0]
    peaks_y2 = np.asarray(peaks2)[:,1]
    fig, ax = plt.subplots(2,3, figsize = (15,10))
    ax = ax.flatten()
    
    ax[0].imshow(imp[0], cmap = 'gray')
    ax[0].scatter(peaks_y1/upsc, peaks_x1/upsc, color = 'red', s = 2)
    ax[1].imshow(imp[1], cmap = 'gray')
    ax[1].set_title('original')
    ax[1].scatter(peaks_y2/upsc, peaks_x2/upsc, color = 'red', s = 2)
    ax[2].imshow(colormap(imp[0], 'magenta') + colormap(imp[1], 'green'))
    
    
    q = []
    for x1, y1, in zip(peaks_x1, peaks_y1):
        for x2, y2 in zip(peaks_x2, peaks_y2):
            if np.linalg.norm([x1-x2, y1-y2]) < dist:
                q.append([x1,y1,x2,y2, np.linalg.norm([x1-x2, y1-y2])])  
    sets_array = np.asarray(q)
    x1, y1, x2, y2, d = np.mean(sets_array, axis = 0)
    
    xd = int(np.round((x1 - x2)/3))
    yd = int(np.round((y1 - y2)/3))
    
    if xd > 0:
        if yd > 0:
            img1 = imp[0][xd:,yd:]
            img2 = imp[1][:-xd,:-yd]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        elif yd < 0:
            img1 = imp[0][xd:,:yd]
            img2 = imp[1][:-xd,-yd:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        else:
            img1 = imp[0][xd:,:]
            img2 = imp[1][:-xd,:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
    
    elif xd < 0:
        if yd > 0:
            img1 = imp[0][:xd,yd:]
            img2 = imp[1][-xd:,:-yd]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        elif yd < 0:
            img1 = imp[0][:xd,:yd]
            img2 = imp[1][-xd:,-yd:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        else:
            img1 = imp[0][:xd,:]
            img2 = imp[1][-xd:,:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))  
    
    else:
        if yd > 0:
            img1 = imp[0][:,yd:]
            img2 = imp[1][:,:-yd]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        elif yd < 0:
            img1 = imp[0][:,:yd]
            img2 = imp[1][:,-yd:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))
        else:
            img1 = imp[0][:,:]
            img2 = imp[1][:,:]
            
            ax[3].imshow(img1, cmap = 'gray')
            ax[4].imshow(img2, cmap = 'gray')
            ax[5].imshow(colormap(img1, 'magenta') + colormap(img2, 'green'))  

    
    ax[4].set_title(f'aligned, shift x: {yd} and y: {xd} pixels with pixel size : {pixel_size}')

    for axn in ax:
        axn.axis('off')

# Run alignment .msr files

In [ ]:
path = Path(f'Y:\\staff\\LowCost\\fse\\smb\\Vlijm\\Frank Mol\\Abberior_STED_RAW\\Aligments_STED_microscopy\\_Oil_Objective')
msr_list = list(path.glob('*.msr'))

In [ ]:
x = []
y = []
m = []
ds = []


for n, alignment_path in enumerate(list(msr_list)):

    date = get_date(alignment_path)
    if date:
        path_test = alignment_path
        try:
            xerr, yerr, mi, date = misalignment(path_test, ['meas30', 'Conf_Overlay-640-560'], disp = True, coord_average = False)
            x.append(xerr)
            y.append(yerr)
            m.append(mi)
            ds.append(date)
            print(f'{date} has {mi:.2f} nm misalignement, with x: {xerr:.2f} nm and y: {yerr:.2f} nm')
        except:
            print(f'no config')
            print(date, str(alignment_path))


# Display errors

In [ ]:
d_avg = []
for xi, yi in zip(x,y):
    d_avg.append(np.linalg.norm([xi,yi]))

fig, ax = plt.subplots(3, 1, figsize = (16, 9))
ax[0].plot(ds, m, label = f'dist (avg) : {np.mean(m):.1f} nm', alpha = .8, color = 'tab:orange')
ax[0].plot(ds, np.ones(len(m))*np.mean(m), alpha = .4, color = 'tab:orange', linestyle = '--')
ax[0].plot(ds, d_avg, label = f'dist (avg x, avg y) : {np.mean(d_avg):.1f} nm', alpha = .8, color = 'tab:blue')
ax[0].plot(ds, np.ones(len(d_avg))*np.mean(d_avg), alpha = .4, color = 'tab:blue', linestyle = '--')
ax[0].plot(ds, np.zeros(len(ds)), color = 'black', linestyle = '--')
ax[0].legend()
ax[0].set_ylabel('Distance (nm)')

ax[1].plot(ds, x, color = 'tab:green', label = 'x error', alpha = .8)
ax[1].plot(ds, np.ones(len(x))*np.mean(x), color = 'tab:green', alpha = .8, linestyle = '--')
ax[1].plot(ds, y, color = 'tab:red', label = 'y error', alpha = .8)
ax[1].plot(ds, np.ones(len(y))*np.mean(y), color = 'tab:red', alpha = .8, linestyle = '--')
ax[1].plot(ds, np.zeros(len(ds)), color = 'black', linestyle = '--')
ax[1].legend()
ax[1].set_ylabel('Distance (nm)')

ax[2].plot(ds, np.abs(x), color = 'tab:green', label = f'abs x error : {np.mean(np.abs(x)):.1f} nm', alpha = .8)
ax[2].plot(ds, np.ones(len(x))*np.mean(np.abs(x)), color = 'tab:green', alpha = .8, linestyle = '--')
ax[2].plot(ds, np.abs(y), color = 'tab:red', label = f'abs y error : {np.mean(np.abs(y)):.1f} nm', alpha = .8)
ax[2].plot(ds, np.ones(len(y))*np.mean(np.abs(y)), color = 'tab:red', alpha = .8, linestyle = '--')
ax[2].plot(ds, np.zeros(len(ds)), color = 'black', linestyle = '--')
ax[2].legend()
ax[2].set_ylabel('Distance (nm)')


plt.savefig('misalignment_error.png')

# Shift images with calculated error to verify:

In [ ]:
path = msr_list[-14:-5][::-1]
for p in path: 
    config =  'Conf_Overlay-640-560'
    align(p, config)